<a href="https://colab.research.google.com/github/EtienneDouillard/Projet_M1/blob/main/Livrables/Cr%C3%A9ation_r%C3%A9seaux_neurones_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Livrable classes de création et d'entraînement des réseaux de neurones.

## Bibliothèques utiles 

In [15]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import itertools

## Classe NewModel

NewModel est la classe qui permet de créer un modèle unique. 

In [16]:
class NewModel:
  def __init__(self):
    #permet de créer un model 
    self.__model = None
    #Connaitre les constante utiles pour le développement d'un model
    self.__couche = []
    self.__lastCouche = None
    self.__nombreCouche = 0
    self.__nombreNeurone = 0
    self.__tailleEntree = 0
    self.__tailleSortie = 0
    self.__compile = None
    self.__nombreNeuroneParCouche = []
    #permet de connaitre la stabilité du model
    self.__modelCreate = False
    self.__input = True
    self.__output = True

  def visualisationModel(self):
        """
        Cette fonction permet de visualiser le model
        """
        if(self.__modelCreate):
            self.__model.summary()
        else:
            return "Le model n'a pas ete cree"

  def createInput(self,size):
        """
        Permet de créer une entrée avec la taille size
        Ajoute dans la liste couche l'entrée créée en premiere position
        """
        if(self.__modelCreate):
            return "\nLe model est créé, il ne peut plus être changé.\n"
        else:
            #input = Embedding(input_dim=size, output_dim=size)
            input = tf.keras.layers.Input((1,50))
            self.__tailleEntree = size
            self.__couche.insert(0, input)
            self.__input = False

  def createOutput(self,size,activation, typeSortie=1, dropoutValue = None):
        """
        Permet de créer une sortie avec la taille size et une fonction d'activation
        Ajoute dans la liste couche la sortie créée en derniere position
        """
        if(self.__modelCreate):
            return ("\nLe model est créé, il ne peut plus être changé.\n")
        elif(self.__input):
            return ("\nIl faut créer une entrée avant de pouvoir créer une sortie\n")
        else :
            output = {
                1 : tf.keras.layers.Dense(size,activation = activation),
                2 : tf.keras.layers.SimpleRNN(size,activation = activation,return_sequences=True),
                3 : tf.keras.layers.GRU(size,activation=activation,return_sequences=True),
                4 : tf.keras.layers.LSTM(size,activation=activation,return_sequences=True),
            }
            self.__tailleSortie = size
            outputs = output.get(typeSortie,-1)
            if(dropoutValue):
                self.__couche.insert(self.__nombreCouche+1,outputs )
                self.__nombreCouche += 1
                self.__couche.insert(self.__nombreCouche, Dropout(dropoutValue))
            else:
                self.__couche.insert(self.__nombreCouche+1, outputs)
            self.__output = False

  def createCouche(self, size, activation, typeCouche, dropoutValue=None):
        """
        Permet de créer une couche de neurone avec la taille size
        Ajoute dans la liste couche la sortie créée en derniere position
        """
        if(self.__modelCreate):
            return "\nLe model est créé, il ne peut plus être changé.\n"
        elif(self.__input):
            return "\nIl faut créer une entrée avant de pouvoir créer une couche\n"
        else:
            couche = {
                1 : tf.keras.layers.Dense(size,activation = activation),
                2 : tf.keras.layers.SimpleRNN(size,activation = activation,return_sequences=True),
                3 : tf.keras.layers.GRU(size,activation=activation,return_sequences=True),
                4 : tf.keras.layers.LSTM(size,activation=activation,return_sequences=True),
            }

            outputs = couche.get(typeCouche,-1)
            self.__nombreCouche += 1
            if(dropoutValue):
                self.__couche.insert(self.__nombreCouche,outputs)
                self.__nombreCouche += 1
                self.__couche.insert(self.__nombreCouche, Dropout(dropoutValue))
            else:
                self.__couche.insert(self.__nombreCouche, outputs)
            self.__nombreNeuroneParCouche.insert(len(self.__nombreNeuroneParCouche),size)

  def createModel(self):
        """
        Permet de créer un model par rapport à la liste de couche
        """
        if(self.__input or self.__output):
            return "L'entree ou la sortie n'a pas ete definie"
        else:
            inputs = self.__couche[0]
            outputs = self.__couche[0]
            for couche in self.__couche[1:]:
                outputs = couche(outputs)
            #self.__model = Sequential(self.__couche)
            self.__modelCreate = True
            self.__model = keras.models.Model(inputs,outputs)

  def compileModel(self):
        """
        Permet de compiler le modele créé
        """
        if(self.__input or self.__output):
          return "L'entree ou la sortie n'a pas ete definie"
        else:
          self.__model.compile(optimizer="Adam",loss="mse",metrics=["mse","mae","accuracy"])

  def getListeCompositionModel(self):
        """
        Affiche la composition du model
        """
        return self.__couche
    
  def getModel(self):
        """
        Permet de donner le modele créé
        """
        if(self.__input or self.__output):
          return "L'entree ou la sortie n'a pas ete definie"
        else:
          return self.__model

## classe TrainingReseau:


L’idée de cette classe est de tester les différentes métriques des modèles puis de sauvegarder les meilleurs modèles à la meilleure époque.


In [18]:
class TrainingReseau:
  """
  Permet de donner le meilleur modèle sur une liste de modele
  """
  def __init__(self,listModel,x_train,y_train,x_valid,y_valid):
      self.__listModel = listModel
      self.__x_train = x_train
      self.__x_valid = x_valid
      self.__y_train = y_train
      self.__y_valid = y_valid

      # Attributs métrics
      self.__loss = math.inf
      self.__mae = math.inf
      self.__mse = math.inf

      #Valeur recherchée
      self.__best_loss = None
      self.__algo_best_loss = None
      self.__indice_best_epoch_loss = None

      self.__best_mae = None
      self.__algo_best_mae = None
      self.__indice_best_epoch_mae = None

      self.__best_mse = None
      self.__algo_best_mse = None
      self.__indice_best_epoch_mse = None

  def loss_and_epoch(self,history):#Loss
        #Plus Loss se rapproche de 0 mieux la prédiction est bonne.
        #Retourne le meilleur Loss et son epoch
       
        if (self.__loss > history.history["loss"][0]):
            self.__loss = history.history["loss"][0]


  def mae_and_epoch(self,history):#mae = mean(abs(y_true - y_pred)
      #Plus Loss se rapproche de 0 mieux la prédiction est bonne.
      #Retourne le meilleur mae et son epoch 

      if (self.__mae > history.history["mae"][0]):
          self.__mae = history.history["mae"][0]

  
  def mse_and_epoch(self,history):#  MSE = mean(square(y_true - y_pred)
  #Plus mse se rapproche de 0 mieux la prédiction est bonne.
      #Retourne le meilleur mse et son epoch 

      if (self.__mse > history.history["mse"][0]):
          self.__mse = history.history["mse"][0]



  def displaySummary(self,number):
    """Permet de visualiser le model"""
    return self.__listModel[number].summary()


  def fit_one(self,model,nbEpoch,callback):
    return model.fit(self.__x_train,self.__y_train,epochs = nbEpoch,callbacks = [callback],verbose=0)

  def bests_algos(self):
    for model in self.__listModel :
        indice = self.__listModel.index(model)
        print("Modèle indice ",index)
        epochMax = 4
        nbEpoch = range(epochMax)
        for i in nbEpoch:
            print((i/epochMax)*100)
            history=self.fit_one(model,1,None)

            #loss
            loss_resultat = self.loss_and_epoch(history)#Trouver le meilleur loss pour une epoch sur l'entrainement du modèle. 
            if (self.__best_loss > loss_resultat):
                self.__best_loss = loss_resultat
                self.__algo_best_loss = indice
                self.__indice_best_epoch_loss = i

            #mae
            mae_resultat=self.mae_and_epoch(history)#Trouver le meilleur mae pour une epoch sur l'entrainement du modèle. 
            if (self.__best_mae > mae_resultat):
                self.__best_mae = mae_resultat
                self.__algo_best_mae = indice
                self.__indice_best_epoch_mae = i

            #mse
            mse_resultat=self.mse_and_epoch(history)
            if (self.__best_mse > mse_resultat):
                self.__best_mse = mse_resultat
                self.__algo_best_mse =indice
                self.__indice_best_epoch_mse = i

  def displayBestAlgo(self):
    return print("\n\nBest Algo \nBest Loss is algo indice :",self.__algo_best_loss, "with loss value =",self.__best_loss,",indice epoch =",self.__indice_best_epoch_loss,"\nBest mae is algo indice:",resultat[3] , "with mae value =",self.__algo_best_mae,
                  "indice epoch =",self.__indice_best_epoch_mae,"\nBest mse is algo indice:",self.__algo_best_mse , "with mse value =",self.__best_mse,"indice epoch =",self.__indice_best_epoch_mse,"\n")

## Classe CreationListeModel

Cette classe est utilisée pour créer les différents modèles, de façon aléatoire ou forcée. Celle-ci utilise les deux dernières pour fonctionner. De ce fait, l’utilisateur a juste à modifier les paramètres d’entrées

In [17]:
class CreationListeModel:
    def __init__(self):
        #self.__Model = ModelReseau().createInput(50)
        self.__listeFinal = [] #liste avec les models construient
        self.__listeModel = [] #liste avec les bases des modèles
        self.__liste = []
        self.__tailleListe = 0
        self.__pre = [] #Permet de récupérer la valeur précédente d'une liste

    def creationCombinaison(self,listeNombreNeuronesParCouche,typeCouche,listeFA,dropout):
        """
        Permet de créer le nombre maximal de combinaison pour la première couche
        """
        etape1 = []
        etape2 = []
        tupleArgCouche = list(itertools.product(listeNombreNeuronesParCouche,listeFA))
        etape1.append(list(itertools.product(tupleArgCouche,typeCouche)))
        etape2.append(list(itertools.product(etape1,dropout)))
        self.__liste = (etape2)
        self.__tailleListe = len(etape2[0])

    def decomposition(self,maximumCouches, indiceListe = 0,listeVal = []):
        """
        Permet de créer une liste des combinaisons de l'arbre à l'aide de la récursivité.
        """
        listePre = list(listeVal)
        if indiceListe < self.__tailleListe:
            listeVal.append(list(self.__liste[0][indiceListe]))
            for fa in ["tanh","linear"]:
              for typeCouche in [1,2,3,4]:
                  lentghNbCouche = range(len(listeVal))
                  model = NewModel()
                  model.createInput(50)
                  for nbCouche in lentghNbCouche:
                      #print("nbNeurone :",listeVal[nbCouche][0][0][0][0])
                      #print("Type FA :",listeVal[nbCouche][0][0][0][1])
                      #print("TypeCouche :",listeVal[nbCouche][0][0][1])
                      #print("Dropout :",listeVal[nbCouche][1])
                      model.createCouche(listeVal[nbCouche][0][0][0][0],listeVal[nbCouche][0][0][0][1],listeVal[nbCouche][0][0][1],listeVal[nbCouche][1]) #createCouche(self, size, activation, typeCouche, dropoutValue = None):
                  model.createOutput(50,fa,typeCouche)
                  model.createModel()
                  model.compileModel()
                  m = model.getModel()
                  self.__listeFinal.append(m)
            if maximumCouches > 1 :
                self.decomposition(maximumCouches-1,0, listeVal)
            self.decomposition(maximumCouches,indiceListe+1,listePre)

    def getListeModel(self):
          """
          renvoie la liste de model
          """
          return self.__listeFinal

    def getLengthListe(self):
      return len(self.__listeFinal)


## Test de création de réseaux de neuronnes 

In [19]:
#print(liste)
maximumCouches = 10
listeNombreNeuronesParCouche = [10,20,50,100]
typeCouche = [1,2,3,4]
listeFA =["tanh","swish"]
dropout = [None,0.2,0.5]
listeModel = CreationListeModel()
listeModel.creationCombinaison(listeNombreNeuronesParCouche,typeCouche,listeFA,dropout)
listeModel.decomposition(maximumCouches)

NameError: ignored

In [20]:
liste = listeModel.getListeModel()

In [21]:
for i in range(len(liste)):
  liste[i].summary()

Model: "model_240"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_244 (InputLayer)       [(None, 1, 50)]           0         
_________________________________________________________________
dense_2400 (Dense)           (None, 1, 10)             510       
_________________________________________________________________
dense_2401 (Dense)           (None, 1, 50)             550       
Total params: 1,060
Trainable params: 1,060
Non-trainable params: 0
_________________________________________________________________
Model: "model_241"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_245 (InputLayer)       [(None, 1, 50)]           0         
_________________________________________________________________
dense_2402 (Dense)           (None, 1, 10)             510       
____________________________________